# Mangaki

On a plein de références :

In [104]:
from collections import Counter

Counter(Reference.objects.values_list('source', flat=True))

Counter({'AniDB': 120,
         'MAL': 11824,
         'VGMdb': 1,
         'Animeka': 57,
         'Manga-News': 17,
         'Icotaku': 1})

## Works ayant le même titre

In [100]:
import pandas as pd

def nunique_nonzero(series):
    return len(set(c for c in series if c))

df = pd.DataFrame(Work.objects.filter(category__slug='anime').values('id', 'title', 'anidb_aid'))
duplicates = df.groupby('title').agg({'id': 'count', 'anidb_aid': nunique_nonzero}).query('id > 1')

print(f"{len(duplicates)} clusters d'anime ont le même titre sur Mangaki")
duplicates.index.name = None
duplicates

45 clusters d'anime ont le même titre sur Mangaki


,id,anidb_aid
Berserk,2,0
Deadman Wonderland,2,1
Desert Punk,3,0
Digimon: The Movie,2,0
Dimension W,3,0
Doraemon,2,0
Dororo,2,0
Fruits Basket,2,1
Gestalt,2,0
Ghost Talker's Daydream,2,0


In [102]:
VALID_MANGAKI_IDS = set(Work.objects.values_list('id', flat=True))
# Non valide signifie que c'est déjà un doublon identifié

## Même titre + des AniDB ID différents (ouf, zéro)

In [113]:
nb_with_anidb_aid = Work.objects.filter(anidb_aid__gt=0).count()
f'{nb_with_anidb_aid} œuvres ont un AniDB ID renseigné'

'321 œuvres ont un AniDB ID renseigné'

In [111]:
anidb_ids = pd.DataFrame(Reference.objects.filter(source='AniDB', work_id__category__slug='anime').values(
    'identifier', 'work_id', 'work_id__title', 'work_id__anidb_aid')).rename(
    columns={'work_id__title': 'title'})
anidb_ids.groupby('title').agg({
    'work_id': 'count',
    'identifier': pd.Series.nunique,
    'work_id__anidb_aid': nunique_nonzero
}).query(
    'identifier != work_id__anidb_aid').sort_values(['identifier', 'work_id'], ascending=(False, False))

,work_id,identifier,work_id__anidb_aid
title,,,
Grandeek,1,1,0


Grandeek a un objet `Reference` AniDB mais pas de champ `anidb_aid` rempli. Ce n'est pas bien grave.

## Même titre + des MAL ID différents

In [119]:
mal_ids = pd.DataFrame(Reference.objects.filter(source='MAL', work_id__category__slug='anime').values(
    'identifier', 'work_id', 'work_id__title')).rename(
    columns={'work_id__title': 'title'})
mal_duplicates = mal_ids.groupby('title').agg({'work_id': 'count', 'identifier': pd.Series.nunique}).query(
    'identifier > 1').sort_values(['identifier', 'work_id'], ascending=(False, False))
print(f'{len(mal_duplicates)} clusters de works ayant le même titre mais des MAL IDs différents')
mal_duplicates.index.name = None
mal_duplicates

14 clusters de works ayant le même titre mais des MAL IDs différents


,work_id,identifier
The Asterisk War: The Academy City on the Water,9,2
Berserk,4,2
Time of Eve,4,2
Digimon: The Movie,3,2
Sorcerer Hunters,3,2
Spice and Wolf II,3,2
Bakuman.,2,2
Doraemon,2,2
Dororo,2,2
Gangsta.,2,2


Par exemple, il y a 4 œuvres Berserk qui ont le même titre mais 2 MAL IDs différents :

In [120]:
mal_ids.query('title == "Berserk"')

,identifier,work_id,title
8516,33,3409,Berserk
10073,32379,15174,Berserk
10585,32379,14516,Berserk
10586,32379,14327,Berserk


Pour ceux-là il faut voir au cas par cas. En fait le MAL ID 33 correspond à Berserk (2016) qui est sûrement moins bien noté que l'original.

# Mangaki + Manami

In [121]:
import json

with open('../../anime-offline-database/anime-offline-database.json') as f:
    manami = json.load(f)

In [122]:
len(manami['data'])

24018

In [123]:
mangaki = Work.objects.filter(category__slug='anime')
mangaki.count()  # Sur Mangaki

10996

In [124]:
manami['data'][42]

{'sources': ['https://anidb.net/anime/8699',
  'https://anilist.co/anime/11755',
  'https://kitsu.io/anime/6588',
  'https://myanimelist.net/anime/11755',
  'https://notify.moe/anime/nX_ItFiiR'],
 'title': '009 Re:Cyborg',
 'type': 'Movie',
 'episodes': 1,
 'status': 'FINISHED',
 'picture': 'https://cdn.myanimelist.net/images/anime/9/40189.jpg',
 'thumbnail': 'https://cdn.myanimelist.net/images/anime/9/40189t.jpg',
 'synonyms': ['009 RE:CYBORG', 'ぜろぜろないん り・さいぼーぐ'],
 'relations': ['https://anidb.net/anime/1214',
  'https://anidb.net/anime/12277',
  'https://anidb.net/anime/1527',
  'https://anidb.net/anime/7994',
  'https://anilist.co/anime/1678',
  'https://anilist.co/anime/4690',
  'https://anilist.co/anime/8394',
  'https://kitsu.io/anime/12326',
  'https://kitsu.io/anime/12397',
  'https://kitsu.io/anime/12410',
  'https://kitsu.io/anime/12543',
  'https://kitsu.io/anime/1508',
  'https://kitsu.io/anime/3741',
  'https://kitsu.io/anime/7868',
  'https://myanimelist.net/anime/1678',


In [125]:
from collections import defaultdict
import re

references = set()
url1 = defaultdict(list)

# Get all sources of Manami entries
for manami_id, entry in enumerate(manami['data']):
    references.add('manami/{}'.format(manami_id))
    for url in entry['sources']:
        ref = re.sub(r'https?://', '', url)
        references.add(ref)
        url1[manami_id].append(ref)
len(references)

94422

In [126]:
url2 = defaultdict(set)

# Mangaki works with AniDB ID
anidb_id = dict(Work.objects.filter(anidb_aid__gt=0).values_list('id', 'anidb_aid'))
for mangaki_id, aid in anidb_id.items():
    ref1 = 'mangaki.fr/work/{}'.format(mangaki_id)
    ref2 = 'anidb.net/anime/{}'.format(aid)
    references.update([ref1, ref2])
    url2[mangaki_id].update([ref1, ref2])

# All Mangaki references
for mangaki_id, url in Reference.objects.values_list('work_id', 'url'):
    ref1 = 'mangaki.fr/work/{}'.format(mangaki_id)
    ref2 = re.sub(r'https?://', '', url).replace('anidb.net/perl-bin/animedb.pl?show=anime&aid=', 'anidb.net/anime/')
    references.update([ref1, ref2])
    url2[mangaki_id].update([ref1, ref2])
len(references)

107203

In [127]:
ids = dict(zip(sorted(references), range(len(references))))

In [128]:
list(ids.keys())[:5]

['anidb.net/anime/1',
 'anidb.net/anime/10',
 'anidb.net/anime/100',
 'anidb.net/anime/1000',
 'anidb.net/anime/10000']

In [129]:
from tryalgo.kruskal import UnionFind

uf = UnionFind(len(references))
nb_merge = 0
for manami_id, refs in url1.items():
    for ref in refs:
        nb_merge += uf.union(ids[ref], ids['manami/{}'.format(manami_id)])
nb_merge

70404

In [137]:
nb_has_anidb = 0
for mangaki_id, refs in url2.items():
    has_anidb = False
    for ref in refs:
        if 'anidb' in ref:
            has_anidb = True
        nb_merge += uf.union(ids[ref], ids['mangaki.fr/work/{}'.format(mangaki_id)])
    nb_has_anidb += has_anidb
f'{nb_merge} fusions : {nb_has_anidb}/{len(url2)} avaient un ID AniDB ({nb_has_anidb/len(url2) * 100:.1f} %)'

'82507 fusions : 322/11976 avaient un ID AniDB (2.7 %)'

In [16]:
clusters = defaultdict(list)
for ref, ref_id in ids.items():
    clusters[uf.find(ref_id)].append(ref)
len(clusters)

24696

In [138]:
from collections import Counter

c = Counter()
clusters_by_occ = defaultdict(list)

nb_cdup_mangaki = 0
nb_cdup_manami = 0
total_ref = 0
search_queries = []
mangaki_not_manami = set()
for cluster in clusters.values():
    mangaki_refs = [ref for ref in cluster if ref.startswith('mangaki') and get_mangaki_id(ref) in VALID_MANGAKI_IDS]
    nb_mangaki = len(mangaki_refs)
    nb_manami = len([ref for ref in cluster if ref.startswith('manami')])
    if nb_mangaki > 0:
        total_ref += len(cluster) - nb_mangaki - nb_manami
    if nb_mangaki >= 2 and nb_manami >= 1:
        nb_cdup_mangaki += 1
    if nb_manami >= 2 and nb_mangaki >= 1:
        nb_cdup_manami += 1
    # if nb_mangaki + nb_manami >= 14:
    #     print(cluster)  # It was Kara no Kyoukai
    if nb_mangaki + nb_manami >= 12:
        print(cluster)
    if nb_mangaki >= 3 or nb_manami >= 3:  # Have to analyze
        clusters_by_occ[nb_mangaki, nb_manami].append(cluster)
    if nb_mangaki == 0 and nb_manami == 1:
        manami_url = [ref for ref in cluster if ref.startswith('manami')][0]
        manami_id = get_manami_id(manami_url)
        search_queries.append(manami['data'][manami_id]['title'])  # In Manami but not in Mangaki
    if nb_manami == 0:
        for url in mangaki_refs:
            mangaki_id = get_mangaki_id(url)
            mangaki_not_manami.add(mangaki_id)
    c[nb_mangaki, nb_manami] += 1
c, nb_cdup_mangaki, nb_cdup_manami, total_ref

['anidb.net/anime/4932', 'kitsu.io/anime/2357', 'kitsu.io/anime/3248', 'kitsu.io/anime/3249', 'kitsu.io/anime/3545', 'kitsu.io/anime/3546', 'manami/5990', 'manami/9868', 'manami/9869', 'manami/9870', 'manami/9871', 'manami/9872', 'manami/9874', 'mangaki.fr/work/1374', 'mangaki.fr/work/1375', 'mangaki.fr/work/1376', 'mangaki.fr/work/1377', 'mangaki.fr/work/1378', 'mangaki.fr/work/1381', 'mangaki.fr/work/14626', 'mangaki.fr/work/14629', 'myanimelist.net/anime/2593', 'myanimelist.net/anime/3782', 'myanimelist.net/anime/3783', 'myanimelist.net/anime/4280', 'myanimelist.net/anime/4282', 'myanimelist.net/anime/5205', 'notify.moe/anime/6AbXcFmiR', 'notify.moe/anime/Pz0l5Fmig', 'notify.moe/anime/RNxu5Kmmg', 'notify.moe/anime/TxDs5KmiR', 'notify.moe/anime/Y8T_cKmmR']


(Counter({(1, 1): 9935,
          (0, 1): 13529,
          (2, 1): 427,
          (1, 2): 29,
          (2, 2): 14,
          (3, 1): 12,
          (5, 1): 1,
          (3, 2): 3,
          (4, 3): 1,
          (8, 7): 1,
          (1, 3): 1,
          (4, 1): 2,
          (3, 4): 1,
          (2, 3): 1,
          (1, 0): 733,
          (2, 0): 6}),
 463,
 51,
 42910)

In [140]:
%%time

# This is long. Don't do this.
'''
search_results = Counter()
print(len(search_queries))
for title in search_queries:
    nb_results = Work.objects.filter(title=title).count()
    search_results[nb_results] += 1
search_results'''
# Counter({0: 12849, 1: 663, 2: 13, 3: 4})

ids_found_in_mangaki = set(Work.objects.filter(title__in=search_queries).values_list('id', flat=True))

CPU times: user 39.5 ms, sys: 11.7 ms, total: 51.2 ms
Wall time: 2.1 s


In [141]:
len(ids_found_in_mangaki)

673

In [143]:
paired = ids_found_in_mangaki & mangaki_not_manami
len(paired)

44

In [145]:
Work.objects.filter(id__in=paired)

<WorkQuerySet [<Work: Azumanga Daioh>, <Work: Sora no Otoshimono>, <Work: Doraemon: Summer Holiday>, <Work: Maria†Holic Alive Special>, <Work: Beelzebub: Sakigake!! Beel to Shinsengumi>, <Work: Hidamari Sketch x Honeycomb Specials>, <Work: Transformers Generation 1>, <Work: Saiunkoku Monogatari Specials>, <Work: Beelzebub: Kaiketsu!! Beel-bo Meitantei Suiri>, <Work: Ginga Hyouryuu Vifam: Atsumatta 13-nin>, <Work: Toilet no Kamisama>, <Work: Manyuu Hikenchou Picture Drama>, <Work: Saru Kani Gassen>, <Work: Chou Seimeitai Transformers Beast Wars Metals: Convoy Daihenshin!>, <Work: Kitakubu Katsudou Kiroku: Miniature Theater>, <Work: One Piece: Romance Dawn>, <Work: One More Time, One More Chance>, <Work: Mazinger Z vs. Dr. Hell>, <Work: Nurse Witch Komugi-chan Magikarte Z Special>, <Work: Gundam Build Fighters OVA>, '...(remaining elements truncated)...']>

In [156]:
nb_in_mangaki_not_manami_hard_to_pair = len(mangaki_not_manami - paired)
print(nb_in_mangaki_not_manami_hard_to_pair, 'to pair')

for work in Work.objects.filter(id__in=mangaki_not_manami - paired):
    if work.worktitle_set.all():
        print(work.worktitle_set.all())
        break

701 to pair
<QuerySet [<WorkTitle: Young Bride's Story (source: mal, type: synonym) attached to A Bride's Story>, <WorkTitle: Otoyome-gatari (source: mal, type: synonym) attached to A Bride's Story>, <WorkTitle: The Bride's Stories (source: mal, type: synonym) attached to A Bride's Story>, <WorkTitle: Pariya-san wa Otoshigoro (source: mal, type: synonym) attached to A Bride's Story>, <WorkTitle: A Bride's Story - en (source: mal, type: main) attached to A Bride's Story>]>


In [38]:
def get_mangaki_id(url):
    return int(url[len('mangaki.fr/work/'):])

def get_manami_id(url):
    return int(url[len('manami/'):])

def translate_url(url):
    if url.startswith('mangaki'):
        mangaki_id = get_mangaki_id(url)
        return f'mangaki:{Work.objects.get(id=mangaki_id).title}'
    if url.startswith('manami'):
        manami_id = get_manami_id(url)
        return f'manami:{manami["data"][manami_id]["title"]}'
    return url

for key in clusters_by_occ:
    print(key)
    for cluster in clusters_by_occ[key]:
        print([translate_url(url) for url in cluster])

In [18]:
Work.all_objects.filter(id=15307)

<WorkQuerySet [<Work: Puella Magi Madoka Magica the Movie Part 2: Eternal>]>

In [19]:
from collections import Counter

c2 = Counter()
nb_with_anidb = 0
nb_c_mangaki = 0
for cluster in clusters.values():
    nb_mangaki = len([ref for ref in cluster if ref.startswith('mangaki')])
    nb_anidb = len([ref for ref in cluster if ref.startswith('anidb')])
    if nb_mangaki > 0:
        nb_c_mangaki += 1
        if nb_anidb > 0:
            nb_with_anidb += 1
    c2[nb_mangaki, nb_anidb] += 1
c2, '{}/{} désormais'.format(nb_with_anidb, nb_c_mangaki)

(Counter({(1, 1): 5384,
          (0, 1): 5414,
          (2, 1): 315,
          (3, 1): 40,
          (4, 1): 8,
          (5, 1): 2,
          (3, 2): 4,
          (4, 2): 1,
          (8, 1): 2,
          (1, 0): 5084,
          (0, 0): 8115,
          (2, 0): 303,
          (3, 0): 18,
          (4, 0): 6}),
 '5756/11167 désormais')

In [21]:
5756/11167

0.5154473000805946

In [22]:
Reference.objects.count()

12020